In [1]:
import pandas as pd
from langgraph.graph import StateGraph
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.language_models import BaseChatModel
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import List, TypedDict
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment
import re
import pdfplumber

# 定義狀態字典類型，用於保存每個客戶的狀態信息
class CustomerState(TypedDict):
    news_content: str  # 輸入的新聞內容
    customer_features: List[str]  # 客戶特徵列表
    extracted_features: str  # 提取的客戶特徵摘要
    news_summary: str  # 基於客戶特徵的新聞摘要
    investment_advice_result: str  # 生成的投資建議
    impact_score: int  # 新增的影響程度分數

# 定義個性化新聞代理類
class PersonalizedNewsAgent:

    def __init__(self, model: BaseChatModel, checkpointer: SqliteSaver):
        self.model = model  # 初始化生成式語言模型
        self.checkpointer = checkpointer  # 初始化檢查點保存器，用於保存狀態
        builder = StateGraph(CustomerState)  # 創建StateGraph並傳入狀態字典類型
        builder.add_node("initialize", self.initialize_node)
        builder.add_node("extract_customer_features", self.extract_customer_features_node)
        builder.add_node("personalized_news_summary", self.personalized_news_summary_node)
        builder.add_node("investment_advice", self.investment_advice_node)
        builder.add_node("impact_assessment", self.impact_assessment_node)  # 新增影響評估節點
        builder.add_node("output_results", self.output_results_node)

        builder.set_entry_point("initialize")
        builder.add_edge("initialize", "extract_customer_features")
        builder.add_edge("extract_customer_features", "personalized_news_summary")
        builder.add_edge("personalized_news_summary", "investment_advice")
        builder.add_edge("investment_advice", "impact_assessment")  # 添加新的邊
        builder.add_edge("impact_assessment", "output_results")
        self.graph = builder.compile(checkpointer=checkpointer)

    def initialize_node(self, state: CustomerState):
        return {
            "extracted_features": "",
            "news_summary": "",
            "investment_advice_result": "",
            "impact_score": 50,  # 初始化影響程度分數為 50（中等）
        }

    def extract_customer_features_node(self, state: CustomerState):
        customer_features_text = "\n".join(state["customer_features"])
        messages = [
            SystemMessage(content="以下所有產品皆為基金類型的產品，請依照以下要求列點總結每位客戶的特徵，並在200字內完成。 請確保生成的內容完全基於以下提供的客戶特徵信息，不參考其他來源。格式需對所有客戶保持一致，並使用繁體中文撰寫。 請按以下格式進行列點總結： 1.投資配置：描述總成交量（數值），主要投資產品類型及比例。 2.風險偏好：P值及C值，簡要描述客戶的風險偏好 3.產品偏好：列出客戶偏好的產品類型 4.持有產品風險分佈：RR值分佈。 5.投資策略：概述客戶的主要投資策略。 6.交易狀況：描述近期的交易頻率和活動。 7.持倉狀況：簡述目前的庫存量，包括產品及數值。 8.總結：綜合以上信息進行簡要總結。 本行各種投資型金融商品風險等級由低至高分為四個等級：P1、P2、P3、P4。 P1等級：適合風險承受度為C1（保守型）及以上的客戶。 P2等級：適合風險承受度為C2（穩健型）及以上的客戶。 P3等級：適合風險承受度為C3（成長型）及以上的客戶。 P4等級：適合風險承受度為C4（積極型）客戶。"),
            HumanMessage(content=customer_features_text),
        ]
        response = self.model.invoke(messages)
        state["extracted_features"] = response.content.strip()
        return {"extracted_features": state["extracted_features"]}

    def personalized_news_summary_node(self, state: CustomerState):
        messages = [
            SystemMessage(content="假設你是銀行的理財專員，基於客戶的以下特徵和新聞內容，生成個人化新聞推薦，需符合以下要求： 僅依據以下提供的客戶特徵及新聞內容，不猜測或假設客戶的其他持有產品或喜好。 生成3至6點新聞推薦，最多6點，且影響性最大的新聞需置於最前，不需要說明影響原因。 每點新聞推薦不超過30字，總字數少於300字。 僅根據以下給定的新聞內容和客戶特徵進行分析，避免外部資訊或假設。 請用繁體中文回答。"),
            HumanMessage(content=f"客戶特徵: {state['extracted_features']}"),
            HumanMessage(content=state["news_content"])
        ]
        response = self.model.invoke(messages)
        state["news_summary"] = response.content.strip()
        return {"news_summary": state["news_summary"]}

    def investment_advice_node(self, state: CustomerState):
        messages = [
            SystemMessage(content="請假設你是銀行的理財專員， 請嚴格基於以下每位客戶特徵和新聞內容，列點生成個人化投資建議，直接給具體建議即可，不要加入其他外部資訊，字數在200字內。投資建議應考慮客戶的風險承受能力、資產配置偏好以及新聞摘要狀況，請用繁體中文回答。"),
            AIMessage(content=f"客戶特徵: {state['extracted_features']}"),
            AIMessage(content=f"新聞摘要: {state['news_summary']}"),
        ]
        response = self.model.invoke(messages)
        state["investment_advice_result"] = response.content.strip()
        return {"investment_advice_result": state["investment_advice_result"]}

    def impact_assessment_node(self, state: CustomerState):
        messages = [
        SystemMessage(content="你是一位具有 CFA Level 3 資格的資深投資專家，請評估新聞對每個客戶的影響程度。給出0到100分的評分，分數越高代表影響越大。"),
        AIMessage(content=f"客戶特徵: {state['extracted_features']}"),
        AIMessage(content=f"新聞摘要: {state['news_summary']}"),
    ]
        response = self.model.invoke(messages)
        impact_text = response.content.strip()

    # 初始化默認值
        score = 20
        explanation = "影響甚小，先給予20分"

    # 使用正則表達式提取数字評分
        score_match = re.search(r'\b(\d{1,3})\b', impact_text)
        if score_match:
            score = int(score_match.group(1))
            score = min(max(score, 0), 100)
            explanation = impact_text[impact_text.find(score_match.group(1)) + len(score_match.group(1)):].strip()
            explanation = explanation.replace('*', '').strip()  # 移除可能的米字號或其他不必要符号

        state["impact_score"] = f"{score} 分"
        return {"impact_score": state["impact_score"]}

    def output_results_node(self, state: CustomerState):
        return {
            "extracted_features": state["extracted_features"],
            "news_summary": state["news_summary"],
            "investment_advice_result": state["investment_advice_result"],
            "impact_score": state["impact_score"],  # 包含影響程度分數
        }

# 定義格式化輸出結果的函數
def format_output_for_customer(customer_name, extracted_features, news_summary, investment_advice_result, impact_score):
    return (f"客戶名稱: {customer_name}\n"
            f"提取的客戶特徵:\n{extracted_features}\n\n"
            f"新聞摘要:\n{news_summary}\n\n"
            f"投資建議:\n{investment_advice_result}\n\n"
            f"新聞對客戶的影響程度: {impact_score} 分\n")

# 定義PDF提取摘要的函數
def extract_pdf_summary(pdf_file_path):
    summary = []
    with pdfplumber.open(pdf_file_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            summary.append(text[:200] + "...")  # 提取前200個字作為摘要
    return "\n".join(summary)

# 初始化模型和檢查點保存器
model = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0)
memory = SqliteSaver.from_conn_string(":memory:")

bot = PersonalizedNewsAgent(model=model, checkpointer=memory)
    
thread = {"configurable": {"thread_id": "1"}}

# 讀取Excel表格
df = pd.read_excel("teste_3.xlsx")

# 手動輸入新聞內容
news_content_input = """
1.美聯儲降息：美聯儲已將利率下調50個基點至4.75-5.0%，這是自2008年金融危機以來的重大舉措​。
2.美元前景：預計美元指數將在100-102之間反彈，隨著其他G10國家進入寬鬆周期，美元仍可能保持強勢​。
3.美國國債收益率：預計10年期美國國債收益率將在3.6%到3.8%之間波動​。
4.股市前景：標普500指數的回購暫停，但企業獲利增長趨勢均衡，有助於第四季度表現​。
5.健護產業股票：隨著美國經濟放緩，醫療保健產業的防禦性被市場重視，成為強勢股票​。
6.金融債券：隨著利差擴大和銀行資本要求的提高，金融債券的吸引力增強​。
7.印度股市：SENSEX指數表現強勁，顯示其有能力應對美國經濟放緩的挑戰​。
8.聯準會政策：聯準會啟動自2020年以來首次降息，且即將修改銀行資本要求​。
9.經濟數據：聯準會預測2024年GDP增長放緩至2%，失業率升至4.4%​。
10.通膨預測：核心PCE預計2026年底降至2%目標，進一步降息空間存在​。
11.美國銀行資本改革：雖然資本要求提高，但調幅較小，有助於穩定金融系統​。
12.8月美國零售銷售：零售數據優於預期，顯示經濟仍有韌性，推動軟著陸的可能性​。
13.日本通膨與升息：東京CPI的升幅增加，使日本央行可能再次升息​。
14.美國新屋銷售：7月創下新高，顯示消費者對未來經濟持樂觀態度​。
15.中東局勢：黎巴嫩爆炸案導致中東緊張局勢升溫，推高油價​。
16.匯市表現：美元表現反彈，主要是因聯準會符合預期的降息決策​。
17.黃金期貨：全球央行和ETF持續增持黃金，顯示市場對黃金的需求強勁​。
18.債市：美國公司債的利差明顯收窄，市場預期降息有助於企業的體質改善​。
19.全球債券市場表現：投資級和非投資級債券的利差有所改善，表現穩定​。
20.首次降息的影響：美國歷史上多次降息後，兩年期公債殖利率均出現下降，特別是在經濟放緩或衰退的情境下​。
21.銀行業利差：隨著通膨數據放緩，銀行業與非金融業的利差逐步收窄，但仍高於歷史平均水準​。
22.標普500表現：因美聯儲降息預期，標普500再創歷史新高，但隨後因套利交易出現震盪​。
23.新興市場：受美元貶值影響，新興市場指數表現走強，特別是印度股市創下新高​。
24.能源板塊：能源股近期表現主要受油價反彈帶動，然而未來仍需關注基本面​。
25.標普500企業盈餘：預計明年企業盈餘增長更加均衡，利好股市發展​。
26.醫療保健類股：醫療保健類股在降息後3個月內有明顯超額報酬，特別是大型生技公司​。
27.印度股市亮點：在非衰退降息時期，印度股市表現優於全球其他市場，並延續其增長趨勢​。
28.投資機會市場：印度、台灣的科技和金融板塊，特別是AI和高速運算需求，提供持續增長機會​。

"""

# 定義輸入的新聞內容
state = {
    "news_content": news_content_input
}

# 生成客戶特徵
all_customers_output = []
all_customers_data = []

# 自動讀取Excel欄位名稱，排除ID欄位
selected_columns = df.columns[1:]

# 迭代每個客戶，生成特徵摘要和投資建議
for index, row in df.iterrows():
    customer_name = row['ID']
    
    # 初始化每位客戶的狀態，確保不受之前狀態的影響
    state = {
        "news_content": news_content_input,  # 保持新聞內容不變
        "customer_features": [],  # 清空之前的客戶特徵
        "extracted_features": "",
        "news_summary": "",
        "investment_advice_result": "",
        "impact_score": 50  # 初始化影響程度分數為 50（中等）
    }
    
    # 根據選擇的欄位來生成客戶特徵
    customer_features = []
    for col in selected_columns:
        if pd.notna(row[col]):  # 檢查該欄位是否有數值
            feature = f"{col}: {row[col]}"
            customer_features.append(feature)
    
    # 更新狀態的客戶特徵
    state["customer_features"] = customer_features

    # 執行圖形流程
    result = bot.graph.invoke(state, thread)

    # 確保投資建議中只包含投資建議的結果，而非新聞摘要
    # 你可以檢查並確保 state["investment_advice_result"] 僅包含投資建議
    if result["news_summary"] in result["investment_advice_result"]:
        result["investment_advice_result"] = result["investment_advice_result"].replace(result["news_summary"], "").strip()

    # 檢查生成的投資建議是否為空，如果是空的，給予警告並記錄
    if not result["investment_advice_result"]:
        print(f"警告: {customer_name} 的投資建議生成為空！")

    # 格式化輸出結果
    customer_output = format_output_for_customer(
        customer_name, 
        result["extracted_features"], 
        result["news_summary"],  # 保留新聞摘要
        result["investment_advice_result"], 
        result["impact_score"]  # 添加影響程度分數
    )
    
    all_customers_output.append(customer_output)

    all_customers_data.append({
        "客戶": customer_name,
        "客戶特徵摘要": result["extracted_features"],
        "新聞摘要": result["news_summary"],  # 保留新聞摘要
        "投資建議": result["investment_advice_result"],  # 只保留投資建議
        "影響程度分數": result["impact_score"]  # 新增的影響程度分數列
    })

# 創建DataFrame並輸出到Excel
output_df = pd.DataFrame(all_customers_data)
with pd.ExcelWriter("Personalized_Investment_Advice_Output_V01.1_2.xlsx", engine="openpyxl") as writer:
    output_df.to_excel(writer, sheet_name="Sheet1", index=False)
    worksheet = writer.sheets["Sheet1"]
    
    # 調整欄位寬度
    for i, col in enumerate(output_df.columns):
        max_width = max(output_df[col].astype(str).map(len).max(), len(col)) + 2
        worksheet.column_dimensions[get_column_letter(i + 1)].width = max_width
    
    # 文字換行
    for row in worksheet.iter_rows():
        for cell in row:
            cell.alignment = Alignment(wrap_text=True)

print("Excel 文件已生成並保存，請檢查結果。")


c:\Users\user\AppData\Local\pypoetry\Cache\virtualenvs\senpai-m2n-YTg9-py3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Excel 文件已生成並保存，請檢查結果。
